In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM
import lifelines
from matplotlib import pyplot as plt
from scipy import stats
from sklearn_pandas import DataFrameMapper
from lime import lime_tabular
import lime
from numpy.random import default_rng
import GPy
random_state = 20
from scipy import stats

In [2]:
data = pd.read_table('../data/brca_metabric_clinical_data.tsv')

In [3]:
data_ = data.drop(['Study ID', 'Patient ID', 'Sample ID', 'Type of Breast Surgery', 'Cancer Type Detailed', 'Cohort'
                  , 'HER2 status measured by SNP6', 'Hormone Therapy', 'Integrative Cluster', 'Oncotree Code', 'Pam50 + Claudin-low subtype'
                  , 'ER status measured by IHC', 'Number of Samples Per Patient', 'Patient\'s Vital Status', 'Radio Therapy'
                   , 'Sex', 'Cancer Type', 'Tumor Stage', 'Sample Type', '3-Gene classifier subtype', 'Tumor Other Histologic Subtype'], axis = 1)

In [4]:
leave_columns = ['Cellularity', 'Chemotherapy', 'ER Status', 'HER2 Status', 
                 'Inferred Menopausal State', 'Primary Tumor Laterality', 'PR Status', 'Neoplasm Histologic Grade']
numerical_columns = ['Age at Diagnosis', 'Lymph nodes examined positive', 'Mutation Count',
                    'Nottingham prognostic index', 'Relapse Free Status (Months)', 'Tumor Size']
labels = ['Overall Survival Status', 'Overall Survival (Months)']

In [5]:
data_ = data_[data_['Overall Survival Status'].notna()]
data_ = data_[data_['Overall Survival (Months)'] > 0]

In [6]:
d = {'0:LIVING': False, '1:DECEASED': True}
data_['Overall Survival Status'] = data_['Overall Survival Status'].map(d)

In [7]:
X_data = data_[numerical_columns+leave_columns]
Y_data = data_[labels]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.20, random_state=random_state)

In [9]:
numer_imputer = [([col], [SimpleImputer(missing_values = np.nan, strategy = 'mean')]) for col in numerical_columns]
col_imputer = [([col], [SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')]) for col in leave_columns]
imputer_mapper = DataFrameMapper(numer_imputer + col_imputer, df_out = True)

In [10]:
X_train = imputer_mapper.fit_transform(X_train)
X_test = imputer_mapper.transform(X_test)
# x_train_double_temp = inference_mapper.fit_transform(X_train)
# categorical_features = [6, 7, 8, 9, 10, 11, 12, 13]
categorical_names = {}
i = 6
for feature in leave_columns:
    le = LabelEncoder()
    le.fit(X_train[feature])
    X_train[feature] = le.transform(X_train[feature])
    X_test[feature] = le.transform(X_test[feature])
    categorical_names[i] = le.classes_
    i += 1

In [11]:
numer_preprocess = [([col], [MinMaxScaler()]) for col in numerical_columns]
leave_preprocess = [([col], [OneHotEncoder()]) for col in leave_columns]
encoder_mapper = DataFrameMapper(numer_preprocess+leave_preprocess, df_out = False)

In [12]:
x_mapper_temp = DataFrameMapper(numer_preprocess+leave_preprocess, df_out = True)
x_temp = x_mapper_temp.fit_transform(X_train)

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

In [13]:
x_train = encoder_mapper.fit_transform(X_train)

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

In [14]:
y_train_final = y_train.to_records(index = False, column_dtypes = {'Overall Survival' : 'u1'})

In [15]:
y_train_t = y_train_final.copy()
# y_train_log_t['Overall Survival (Months)'] = np.log1p(y_train_final['Overall Survival (Months)'])

In [16]:
ref_estimator = FastSurvivalSVM(rank_ratio=0.0, max_iter=1000, tol=1e-5, random_state=random_state)
ref_estimator.fit(x_train, y_train_t)

cindex = concordance_index_censored(
    y_train_final['Overall Survival Status'],
    y_train_final['Overall Survival (Months)'],
    -ref_estimator.predict(x_train),  # flip sign to obtain risk scores
)
print(round(cindex[0], 3))

0.916


In [17]:
y_test_final = y_test.to_records(index = False, column_dtypes = {'Overall Survival' : 'u1'})
y_test_final
x_test = encoder_mapper.transform(X_test)

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

In [18]:
# pred_y = np.expm1(ref_estimator.predict(x_test))
cindex = concordance_index_censored(
    y_test_final['Overall Survival Status'],
    y_test_final['Overall Survival (Months)'],
    -ref_estimator.predict(x_test),  # flip sign to obtain risk scores
)
print(round(cindex[0], 3))

0.895


In [19]:
categorical_features = [6, 7, 8, 9, 10, 11, 12, 13]
def predict_fn(x):
    df = pd.DataFrame(x, columns = numerical_columns+leave_columns)
    return ref_estimator.predict(encoder_mapper.transform(df))

In [20]:
# class Surv_XAI:
    
#     def __init__(self, sample_type = "lime", **kwargs):
#         if sample_type == "lime":
#             self.sampler = lime_tabular.LimeTabularExplainer(**kwargs)
#         self.sample_type = sample_type
    
#     def _lime_tabular_perturb_n_samples(self, data, n_samples):
#         """Generates n_perturbations for LIME."""
#         if self.sample_type == "lime":
#             rdata, inverse = self.sampler._LimeTabularExplainer__data_inverse(data_row=data, 
#                                                                             num_samples=n_samples)
#         scaled_data = (rdata - self.sampler.scaler.mean_) / self.sampler.scaler.scale_
#         distances = sklearn.metrics.pairwise_distances(
#             scaled_data,
#             scaled_data[0].reshape(1, -1),
#             metric='euclidean'
#         ).ravel()
#         return rdata, inverse, scaled_data, distances
    
#     def fit_censored_gp(self, X_, y_, cens_):
#         kernel = GPy.kern.Matern52(X_.shape[1],ARD=True) + GPy.kern.White(2)
#         likelihood = GPy.likelihoods.CensoredGaussian(censoring=cens_, variance=1.)
#         gp = GPy.models.GPCensoredRegression(X=X_, Y=y_.reshape(-1,1), censoring=cens_, kernel=kernel, likelihood=likelihood)
#         gp.optimize(optimizer="BFGS", max_iters=2500, messages=False)
#         return gp
    
#     def censored_focussed_sampling(self, point, predict_fn, S, N, A, batch_size, p1, p2):
#         temperature = 1e-2
#         rng = default_rng(random_state)
#         '''
#         1st Step - Generate initial perturbations and censoring labels
#         '''
#         data, data_inverse, scaled_data, distances_ = self._lime_tabular_perturb_n_samples(point, S)
#         yss_pred = np.log(predict_fn(data_inverse))
#         data_rand_cens = rng.choice([0, 1], size = S, p = [p1, p2])
#         data_rand_cens[0] = 0
#         '''
#         2nd Step - Fit the model on initial data
#         '''
#         gp = self.fit_censored_gp(data, yss_pred, data_rand_cens)
#         '''
#         3rd focussed sampling procedure
#         '''
#         for sample_no in range(S, N, batch_size):
#             rdata, inverse, scaled_data, distances = self._lime_tabular_perturb_n_samples(point, A+1)
#             rdata = np.delete(rdata, (0), axis=0)
#             inverse = np.delete(inverse, (0), axis = 0)
#             distances = np.delete(distances, (0), axis = 0)
#             scaled_data = np.delete(scaled_data, (0),  axis = 0)
#             x_samples = gp.posterior_samples_f(X=rdata, size=256)
#             var = np.var(x_samples, axis = 2).flatten()
#             if np.isnan(np.sum(var)):
#                 print("Yes !!!!! Here NAN values come : var")
#             if np.isnan(np.sum(np.abs(distances))):
#                 print("Yes !!!!! Here NAN values come : distances")
#             var = np.abs(var.flatten() - np.abs(distances))
#             var /= temperature
# #             var = var - np.max(var)
#             if np.isnan(np.sum(var)):
#                 print("Yes !!!!! Here NAN values come : var_dist")
#             exp_var = np.exp(var)
#             if np.isnan(np.sum(exp_var)):
#                 print("Yes !!!!! Here NAN values come : exp_var")
#             all_exp = np.sum(exp_var)
#             if np.isnan(np.sum(all_exp)):
#                 print("Yes !!!!! Here NAN values come : all_exp")
#             temperature_scaled_weights = exp_var / all_exp
#             if np.isnan(np.sum(temperature_scaled_weights)):
#                 print("Yes !!!!! Here NAN values come : temp")
#             least_confident_sample = rng.choice(len(var), size=batch_size, p=temperature_scaled_weights, replace=False)
#             yss = np.log(predict_fn(inverse[least_confident_sample]))
#             data = np.concatenate([data, rdata[least_confident_sample]])
#             data_inverse = np.concatenate([data_inverse, inverse[least_confident_sample]])
#             yss_pred = np.concatenate([yss_pred, yss])
#             rand_cens = rng.choice([0, 1], size = batch_size, p = [p1, p2])
#             data_rand_cens = np.concatenate([data_rand_cens, rand_cens])
#             distances_ = np.concatenate([distances_, distances])
#             gp = self.fit_censored_gp(data, yss_pred, data_rand_cens)
#         return gp, data, yss_pred, data_rand_cens
    

In [21]:
# class Surv_XAI:
    
#     def __init__(self, sample_type = "lime", **kwargs):
#         if sample_type == "lime":
#             self.sampler = lime_tabular.LimeTabularExplainer(**kwargs)
#         self.sample_type = sample_type
    
#     def _lime_tabular_perturb_n_samples(self, data, n_samples):
#         """Generates n_perturbations for LIME."""
#         if self.sample_type == "lime":
#             rdata, inverse = self.sampler._LimeTabularExplainer__data_inverse(data_row=data, 
#                                                                             num_samples=n_samples)
#         scaled_data = (rdata - self.sampler.scaler.mean_) / self.sampler.scaler.scale_
#         distances = sklearn.metrics.pairwise_distances(
#             scaled_data,
#             scaled_data[0].reshape(1, -1),
#             metric='euclidean'
#         ).ravel()
#         return rdata, inverse, scaled_data, distances
    
#     def fit_censored_gp(self, X_, y_, cens_):
#         kernel = GPy.kern.Matern52(X_.shape[1],ARD=True) + GPy.kern.White(2)
#         likelihood = GPy.likelihoods.CensoredGaussian(censoring=cens_, variance=1.)
#         gp = GPy.models.GPCensoredRegression(X=X_, Y=y_.reshape(-1,1), censoring=cens_, kernel=kernel, likelihood=likelihood)
#         gp.optimize(optimizer="BFGS", max_iters=2500, messages=False)
#         return gp
    
#     def fit_separate_gp(self, data, new_data, yss_pred, yss, data_rand_cens, rand_cens):
#         data = np.concatenate([data, new_data])
#         yss_pred = np.concatenate([yss_pred, yss])
#         data_rand_cens = np.concatenate([data_rand_cens, rand_cens])
#         return self.fit_censored_gp(data, yss_pred, data_rand_cens)
    
#     def softmax(self, var, temperature = 1e-2):
#         var /= temperature
#         var = var - np.max(var)
#         exp_var = np.exp(var)
#         all_exp = np.sum(exp_var)
#         temperature_scaled_weights = exp_var / all_exp
#         return temperature_scaled_weights
    
#     def fill_censored(self, uncensored_list, cens_var, size):
#         rng = default_rng(random_state)
#         initial_list = range(cens_var.shape[0])
#         diff = list(set(initial_list).difference(set(list(uncensored_list))))
#         return rng.choice(diff, size = size, p = self.softmax(cens_var[diff]), replace = False)
        
#     def fill_uncensored(self, censored_list, uncens_var, size):
#         rng = default_rng(random_state)
#         initial_list = range(uncens_var.shape[0])
#         diff = list(set(initial_list).difference(set(list(censored_list))))
#         return rng.choice(diff, size = size, p = self.softmax(uncens_var[diff]), replace = False)
        
#     def censored_focussed_sampling(self, point, predict_fn, S, N, A, batch_size, p1, p2, filling_type = "censored"):
#         temperature = 1e-2
#         rng = default_rng(random_state)
#         '''
#         1st Step - Generate initial perturbations and censoring labels
#         '''
#         data, data_inverse, scaled_data, distances_ = self._lime_tabular_perturb_n_samples(point, S)
#         yss_pred = np.log(predict_fn(data_inverse))
#         data_rand_cens = rng.choice([0, 1], size = S, p = [p1, p2])
#         data_rand_cens[0] = 0
#         '''
#         2nd Step - Fit the model on initial data
#         '''
#         gp = self.fit_censored_gp(data, yss_pred, data_rand_cens)
#         '''
#         3rd focussed sampling procedure
#         '''
#         for sample_no in range(S, N, batch_size):
#             rdata, inverse, scaled_data, distances = self._lime_tabular_perturb_n_samples(point, A+1)
#             rdata = np.delete(rdata, (0), axis=0)
#             inverse = np.delete(inverse, (0), axis = 0)
#             distances = np.delete(distances, (0), axis = 0)
#             scaled_data = np.delete(scaled_data, (0),  axis = 0)
#             yss = np.log(predict_fn(inverse))
#             uncens_var = []
#             cens_var = []
#             for i in range(rdata.shape[0]):
#                 #uncensored
#                 uncens_gp = self.fit_separate_gp(data, np.expand_dims(rdata[i], axis = 0), yss_pred, np.expand_dims(yss[i], axis = 0), data_rand_cens, np.array([0]))
#                 x_samples = uncens_gp.posterior_samples_f(np.expand_dims(rdata[i], axis = 0), size = 256)
#                 uncens_var.append(np.var(x_samples, axis = 2)[0][0])
                
#                 #censored
#                 cens_gp = self.fit_separate_gp(data, np.expand_dims(rdata[i], axis = 0), yss_pred, np.expand_dims(yss[i], axis = 0), data_rand_cens, np.array([1]))
#                 x_samples = cens_gp.posterior_samples_f(np.expand_dims(rdata[i], axis = 0), size = 256)
#                 cens_var.append(np.var(x_samples, axis = 2)[0][0])
#             cens_var = np.array(cens_var)
#             uncens_var = np.array(uncens_var)
#             cens_var = np.abs(cens_var - distances)
#             uncens_var = np.abs(uncens_var - distances)
#             print(cens_var)
#             print(uncens_var)
# #             cens_var = self.softmax(cens_var)
# #             uncens_var = self.softmax(uncens_var)
#             uncensored_list = None
#             censored_list = None
# #             print(len(uncens_var))
#             if filling_type == "censored":
#                 uncens_var = self.softmax(uncens_var)
#                 uncensored_list = rng.choice(len(uncens_var), size = int(p1*batch_size), p = uncens_var, replace = False)
#                 censored_list = self.fill_censored(uncensored_list, cens_var, int(p2*batch_size))
#             else:
#                 cens_var = self.softmax(cens_var)
#                 censored_list = rng.choice(len(cens_var), size = int(p2*batch_size), p = cens_var, replace = False)
#                 uncensored_list = self.fill_uncensored(censored_list, uncens_var, int(p1*batch_size))
#             least_confident_sample = list(censored_list)+list(uncensored_list)
#             data = np.concatenate([data, rdata[least_confident_sample]])
#             data_inverse = np.concatenate([data_inverse, inverse[least_confident_sample]])
#             yss_pred = np.concatenate([yss_pred, yss[least_confident_sample]])
#             rand_cens = np.array([1]*(int(p2*batch_size)) + [0]*(int(p1*batch_size)))
#             data_rand_cens = np.concatenate([data_rand_cens, rand_cens])
#             gp = self.fit_censored_gp(data, yss_pred, data_rand_cens)
#         return gp, data, yss_pred, data_rand_cens

In [20]:
'''
utils.py
'''


from scipy.stats import rankdata 

def get_scores(xai_model, features, model_type):
    num_features = len(features)
    scores = []
    if model_type == 'gp':
        para = xai_model[:]
        for i in range(1, num_features+1):
            scores.append(para[i])
    elif model_type == "lime":
        para = xai_model.local_exp[0]
        for i in range(num_features):
            for tup in para:
                if tup[0] == i:
                    scores.append(np.abs(tup[1]))
    elif model_type == "weibull":
        for feature in features:
            scores.append(np.abs(xai_model.params_["lambda_"][feature]))
    elif model_type == "lr":
        para = xai_model.coef_
        for i in range(num_features):
            scores.append(np.abs(para[i]))
    return scores

def get_ranks(scores):
    return rankdata(-np.array(scores))

def get_topk_features(scores, k):
    ind = np.argpartition(scores, -k)[-k:]
    return ind


In [21]:
'''
Metrics
'''
def jaccard_index(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def max_sens():
    pass

def deletion(point, predict_fn, rankings):
    point = np.expand_dims(point, 0)
    predictions = [predict_fn(point)]
    point_ = point.copy()
    for feature in rankings:
        point_[:, feature] = 0
        predictions.append(predict_fn(point_))
    return np.abs(np.array(predictions)-predict_fn(point))

def insertion(point, predict_fn, rankings):
    point = np.expand_dims(point, 0)
    predictions = []
    point_ = np.zeros(point.shape)
    for feature in rankings:
        point_[:, feature] = point[:, feature]
        predictions.append(predict_fn(point_))
    predictions.append(predict_fn(point))
    return np.abs(np.array(predictions)-predict_fn(point))

In [22]:
from lifelines import WeibullAFTFitter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
class Surv_XAI_weibull:
    
    def __init__(self, features, X_train, cens_label, sample_type, **kwargs):
        if sample_type == "lime":
            self.sampler = lime_tabular.LimeTabularExplainer(**kwargs)
        self.sample_type = sample_type
        
        self.features = {}
        for i in range(len(features)):
            self.features[i] = features[i]
        
        self.X_train = X_train
        self.cens_label = cens_label
        
#         self.clf = LogisticRegression(random_state = 0).fit(self.X_train, self.cens_label)
#         self.clf = SVC(gamma='auto').fit(self.X_train, self.cens_label)
#         self.clf = DecisionTreeClassifier(random_state=0).fit(self.X_train, self.cens_label)
        self.clf = clf = RandomForestClassifier(max_depth=3, random_state=0).fit(self.X_train, self.cens_label)
        print(f1_score(self.cens_label, self.clf.predict(self.X_train)))
        
    def kernel(self, d, kernel_width = 3):
        return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))
    
    def get_censorship(self, data):
        df = pd.DataFrame(data, columns = numerical_columns+leave_columns)
        return self.clf.predict(encoder_mapper.transform(df))
    
    def _lime_tabular_perturb_n_samples(self, data, n_samples):
        """Generates n_perturbations for LIME."""
        if self.sample_type == "lime":
            rdata, inverse = self.sampler._LimeTabularExplainer__data_inverse(data_row=data, 
                                                                            num_samples=n_samples)
        scaled_data = (rdata - self.sampler.scaler.mean_) / self.sampler.scaler.scale_
        distances = sklearn.metrics.pairwise_distances(
            scaled_data,
            scaled_data[0].reshape(1, -1),
            metric='euclidean'
        ).ravel()
        return rdata, inverse, scaled_data, distances
    
    def fit_weibull(self, point, predict_fn, N, p1, p2):
        data, data_inverse, scaled_data, distances_ = self._lime_tabular_perturb_n_samples(point, N)
        yss_pred = predict_fn(data_inverse)
        df = pd.DataFrame()
        kern_dist = self.kernel(distances_)
        data = np.multiply(data, kern_dist[:, np.newaxis])
        for i in range(data.shape[1]):
            df[self.features[i]] = data[:, i]
        df["y"] = yss_pred*kern_dist
#         rng = default_rng(random_state)
#         df["event"] = rng.choice([0, 1], size = N, p = [p1, p2])
        df["event"] = self.get_censorship(data_inverse)
        weibull_aft = WeibullAFTFitter()
        weibull_aft.fit(df, duration_col='y', event_col='event')
        return weibull_aft 
# weibull_aft = WeibullAFTFitter()
# weibull_aft.fit(data, duration_col='time', event_col='status')
# weibull_aft.print_summary(3)

In [77]:
from lifelines import WeibullAFTFitter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, HuberRegressor, RANSACRegressor, TheilSenRegressor, QuantileRegressor
class Generalized_XAI:
    
    def __init__(self, features, cat_feat, X_train, cens_label, sample_type, **kwargs):
        if sample_type == "lime":
            self.sampler = lime_tabular.LimeTabularExplainer(**kwargs)
        self.sample_type = sample_type
        
        self.features = {}
        for i in range(len(features)):
            self.features[i] = features[i]
        
        self.X_train = X_train
        self.cens_label = cens_label
        
        self.cat_feat = cat_feat
        
#         self.clf = LogisticRegression(random_state = 0).fit(self.X_train, self.cens_label)
#         self.clf = SVC(gamma='auto').fit(self.X_train, self.cens_label)
#         self.clf = DecisionTreeClassifier(random_state=0).fit(self.X_train, self.cens_label)
#         self.clf = clf = RandomForestClassifier(max_depth=3, random_state=0).fit(self.X_train, self.cens_label)
#         print(f1_score(self.cens_label, self.clf.predict(self.X_train)))
        
    def kernel(self, d, kernel_width = 3):
        return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))
    
    def get_censorship(self, data):
        df = pd.DataFrame(data, columns = numerical_columns+leave_columns)
        return self.clf.predict(encoder_mapper.transform(df))
    
    def _lime_tabular_perturb_n_samples(self, data, n_samples):
        """Generates n_perturbations for LIME."""
        if self.sample_type == "lime":
            rdata, inverse = self.sampler._LimeTabularExplainer__data_inverse(data_row=data, 
                                                                            num_samples=n_samples)
        scaled_data = (rdata - self.sampler.scaler.mean_) / self.sampler.scaler.scale_
        distances = sklearn.metrics.pairwise_distances(
            scaled_data,
            scaled_data[0].reshape(1, -1),
            metric='euclidean'
        ).ravel()
        return rdata, inverse, scaled_data, distances
    
    def fit_xai_model(self, point, predict_fn, N, p1, p2):
        data, data_inverse, scaled_data, distances_ = self._lime_tabular_perturb_n_samples(point, N+1)
        index_sample = data[0]
        data = np.delete(data, (0), axis=0)
        data_inverse = np.delete(data_inverse, (0), axis = 0)
        distances_ = np.delete(distances_, (0), axis = 0)
        scaled_data = np.delete(scaled_data, (0),  axis = 0)
        yss_pred = predict_fn(data_inverse)
        yss_index = predict_fn(np.expand_dims(point, 0))
        yss_pred = np.log(yss_index/yss_pred)
        data = np.expand_dims(index_sample, 0) - data
        weights = self.kernel(distances_)
        print(weights)
        for feat in self.cat_feat:
            data[:, feat] = np.abs(data[:, feat])
        xai_model = QuantileRegressor().fit(data, yss_pred, sample_weight = weights)
        return xai_model

In [78]:
# xai_surv = Surv_XAI(training_data = X_train.to_numpy() ,feature_names = numerical_columns+leave_columns,
#                                                    categorical_features=categorical_features, 
#                                                    categorical_names=categorical_names, kernel_width=3, mode='regression', feature_selection = 'none', random_state = random_state, sample_type = "lime")


cens_label = y_train['Overall Survival Status']
d = {False : 1, True : 0}
cens_label = cens_label.map(d)
cens_label.to_numpy()
generalized_xai = Generalized_XAI(features = numerical_columns+leave_columns, cat_feat = [6, 7, 8, 9, 10, 11, 12, 13], X_train = x_train, cens_label = cens_label.to_numpy(), sample_type = "lime", training_data = X_train.to_numpy() ,feature_names = numerical_columns+leave_columns,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3, mode='regression', feature_selection = 'none', random_state = random_state)

In [79]:
# jaccard_dist_mean_ind = []
# jaccard_dist_mean = []
# unique, counts = np.unique(y_train['Overall Survival Status'], return_counts=True)
# p1 = counts[0]/(counts[0]+counts[1])
# p2 = counts[1]/(counts[0]+counts[1])
# k = 1
# index = 1
# point = X_test.to_numpy()[index]
# xai_model = xai_surv.fit_weibull(point, predict_fn, 100, 0.6, 0.4)

import time
from itertools import combinations
# from sklearn.linear_model import LinearRegression, HuberRegressor, TheilSenRegressor
# batch_size = 10
# A = 100
# S = 20
jaccard_dist_mean_ind = []
jaccard_dist_mean = []
unique, counts = np.unique(y_train['Overall Survival Status'], return_counts=True)
p1 = counts[0]/(counts[0]+counts[1])
p2 = counts[1]/(counts[0]+counts[1])
k = 50
num_samples = [100]
# for index in [1]:
for index in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    start_time = time.time()
# for index in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    print("index : "+str(index))
    point = X_test.to_numpy()[index]
    xai_functions_dict = {}
    for N in num_samples:
        print("N : " + str(N))
        xai_functions_dict[N] = {}
        for k_ in range(k):
            print("k : "+str(k_))
            gp = generalized_xai.fit_xai_model(point, predict_fn, N, p1, p2)
            if k_ not in xai_functions_dict[N]:
                xai_functions_dict[N][k_] = [gp]
            else:
                xai_functions_dict[N][k_].append(gp)
    print("--- %s seconds ---" % (time.time() - start_time))
    xai_functions = []
    for key in xai_functions_dict[100]:
        xai_functions.append(xai_functions_dict[100][key][0])
    comb = list(combinations(xai_functions, 2))
    jaccard_dist = []
    for tup in comb:
        scores1 = get_scores(tup[0], numerical_columns+leave_columns, "lr")
        scores2 = get_scores(tup[1], numerical_columns+leave_columns, "lr")
        topk_1 = get_topk_features(scores1, 5)
        topk_2 = get_topk_features(scores2, 5)
        jacc_ind = jaccard_index(topk_1, topk_2)
        jacc_dist = 1-jacc_ind
        jaccard_dist.append(jacc_dist)
        jaccard_dist_mean.append(jacc_dist)
    jaccard_dist_mean_ind.append(np.mean(jaccard_dist))

index : 1
N : 100
k : 0
[0.71653131 0.67780958 0.64118039 0.67780958 0.67780958 0.75746513
 0.64118039 0.84648172 0.71653131 0.71653131 0.64118039 0.64118039
 0.64118039 0.71653131 0.60653066 0.64118039 0.64118039 0.60653066
 0.71653131 0.64118039 0.64118039 0.71653131 0.67780958 0.64118039
 0.71653131 0.67780958 0.71653131 0.75746513 0.75746513 0.67780958
 0.67780958 0.67780958 0.67780958 0.67780958 0.75746513 0.60653066
 0.75746513 0.67780958 0.64118039 0.64118039 0.60653066 0.60653066
 0.75746513 0.64118039 0.64118039 0.64118039 0.67780958 0.64118039
 0.64118039 0.60653066 0.67780958 0.57375342 0.67780958 0.60653066
 0.60653066 0.75746513 0.67780958 0.60653066 0.84648172 0.64118039
 0.60653066 0.60653066 0.60653066 0.60653066 0.67780958 0.84648172
 0.71653131 0.57375342 0.60653066 0.64118039 0.71653131 0.8007374
 0.60653066 0.75746513 0.54274748 0.67780958 0.71653131 0.71653131
 0.64118039 0.75746513 0.60653066 0.67780958 0.64118039 0.67780958
 0.67780958 0.60653066 0.8007374  0.606

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

k : 2
[0.67780958 0.60653066 0.75746513 0.67780958 0.64118039 0.67780958
 0.64118039 0.71653131 0.60653066 0.60653066 0.67780958 0.75746513
 0.60653066 0.71653131 0.67780958 0.67780958 0.64118039 0.64118039
 0.64118039 0.75746513 0.67780958 0.64118039 0.71653131 0.64118039
 0.75746513 0.64118039 0.67780958 0.60653066 0.71653131 0.64118039
 0.67780958 0.60653066 0.75746513 0.67780958 0.71653131 0.64118039
 0.75746513 0.67780958 0.67780958 0.67780958 0.57375342 0.67780958
 0.67780958 0.64118039 0.60653066 0.67780958 0.71653131 0.64118039
 0.67780958 0.67780958 0.60653066 0.60653066 0.67780958 0.71653131
 0.71653131 0.75746513 0.64118039 0.57375342 0.57375342 0.57375342
 0.64118039 0.57375342 0.54274748 0.67780958 0.67780958 0.57375342
 0.67780958 0.67780958 0.75746513 0.71653131 0.64118039 0.57375342
 0.71653131 0.60653066 0.64118039 0.75746513 0.67780958 0.67780958
 0.75746513 0.67780958 0.75746513 0.75746513 0.60653066 0.71653131
 0.60653066 0.67780958 0.75746513 0.71653131 0.84648172 

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

k : 4
[0.60653066 0.75746513 0.71653131 0.71653131 0.8007374  0.71653131
 0.67780958 0.60653066 0.75746513 0.64118039 0.57375342 0.64118039
 0.60653066 0.67780958 0.60653066 0.75746513 0.64118039 0.67780958
 0.67780958 0.67780958 0.64118039 0.67780958 0.57375342 0.64118039
 0.64118039 0.64118039 0.75746513 0.64118039 0.75746513 0.57375342
 0.64118039 0.67780958 0.57375342 0.48567179 0.64118039 0.75746513
 0.67780958 0.67780958 0.64118039 0.60653066 0.64118039 0.67780958
 0.64118039 0.64118039 0.67780958 0.67780958 0.51341712 0.75746513
 0.71653131 0.71653131 0.71653131 0.64118039 0.64118039 0.67780958
 0.71653131 0.75746513 0.75746513 0.64118039 0.57375342 0.60653066
 0.64118039 0.64118039 0.75746513 0.67780958 0.64118039 0.71653131
 0.67780958 0.57375342 0.57375342 0.67780958 0.8007374  0.64118039
 0.75746513 0.71653131 0.75746513 0.71653131 0.60653066 0.54274748
 0.67780958 0.67780958 0.64118039 0.64118039 0.75746513 0.60653066
 0.64118039 0.71653131 0.67780958 0.64118039 0.64118039 

...

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

[0.67780958 0.71653131 0.75746513 0.67780958 0.60653066 0.75746513
 0.67780958 0.75746513 0.67780958 0.64118039 0.51341712 0.71653131
 0.60653066 0.75746513 0.57375342 0.57375342 0.64118039 0.71653131
 0.67780958 0.60653066 0.64118039 0.67780958 0.64118039 0.64118039
 0.71653131 0.60653066 0.64118039 0.57375342 0.64118039 0.8007374
 0.67780958 0.54274748 0.57375342 0.67780958 0.67780958 0.60653066
 0.54274748 0.67780958 0.8007374  0.60653066 0.57375342 0.60653066
 0.71653131 0.57375342 0.71653131 0.71653131 0.71653131 0.71653131
 0.64118039 0.60653066 0.71653131 0.60653066 0.67780958 0.67780958
 0.60653066 0.75746513 0.67780958 0.75746513 0.57375342 0.67780958
 0.67780958 0.60653066 0.57375342 0.57375342 0.67780958 0.64118039
 0.8007374  0.57375342 0.64118039 0.64118039 0.67780958 0.71653131
 0.75746513 0.67780958 0.67780958 0.54274748 0.60653066 0.64118039
 0.60653066 0.71653131 0.54274748 0.60653066 0.60653066 0.8007374
 0.54274748 0.64118039 0.60653066 0.64118039 0.67780958 0.716531

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

[0.75746513 0.71653131 0.64118039 0.64118039 0.67780958 0.57375342
 0.8007374  0.67780958 0.71653131 0.60653066 0.64118039 0.64118039
 0.57375342 0.60653066 0.64118039 0.57375342 0.67780958 0.71653131
 0.71653131 0.60653066 0.67780958 0.60653066 0.60653066 0.57375342
 0.64118039 0.75746513 0.67780958 0.67780958 0.57375342 0.67780958
 0.67780958 0.60653066 0.60653066 0.57375342 0.60653066 0.57375342
 0.64118039 0.60653066 0.71653131 0.57375342 0.64118039 0.64118039
 0.60653066 0.64118039 0.67780958 0.57375342 0.71653131 0.57375342
 0.71653131 0.8007374  0.64118039 0.64118039 0.64118039 0.60653066
 0.75746513 0.60653066 0.60653066 0.64118039 0.67780958 0.60653066
 0.60653066 0.71653131 0.60653066 0.71653131 0.64118039 0.64118039
 0.57375342 0.71653131 0.60653066 0.64118039 0.67780958 0.60653066
 0.57375342 0.67780958 0.64118039 0.64118039 0.67780958 0.64118039
 0.71653131 0.64118039 0.57375342 0.64118039 0.60653066 0.60653066
 0.64118039 0.64118039 0.64118039 0.71653131 0.71653131 0.6411

KeyboardInterrupt: 

In [76]:
np.mean(jaccard_dist_mean_ind)

0.6810048590864917

In [289]:
# import time
# from itertools import combinations
# batch_size = 10
# A = 100
# S = 20
# jaccard_dist_mean_ind = []
# jaccard_dist_mean = []
# unique, counts = np.unique(y_train['Overall Survival Status'], return_counts=True)
# p1 = counts[0]/(counts[0]+counts[1])
# p2 = counts[1]/(counts[0]+counts[1])
# k = 50

# # for index in [1]:
# #     start_time = time.time()
# for index in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
#     print("index : "+str(index))
#     point = X_test.to_numpy()[index]
#     xai_functions_dict = {}
#     for N in [100]:
#         print("N : " + str(N))
#         xai_functions_dict[N] = {}
#         for k_ in range(k):
#             print("k : "+str(k_))
#             gp, _, _, _ = xai_surv.censored_focussed_sampling(point, predict_fn, S, N, A, batch_size, p1, p2)
#             if k_ not in xai_functions_dict[N]:
#                 xai_functions_dict[N][k_] = [gp]
#             else:
#                 xai_functions_dict[N][k_].append(gp)
#     print("--- %s seconds ---" % (time.time() - start_time))
#     xai_functions = []
# for key in xai_functions_dict[100]:
#     xai_functions.append(xai_functions_dict[100][key][0])
# comb = list(combinations(xai_functions, 2))
# jaccard_dist = []
# for tup in comb:
#     scores1 = get_scores(tup[0])
#     scores2 = get_scores(tup[1])
#     topk_1 = get_topk_features(scores1, 5)
#     topk_2 = get_topk_features(scores2, 5)
#     jacc_ind = jaccard_index(topk_1, topk_2)
#     jacc_dist = 1-jacc_ind
#     jaccard_dist.append(jacc_dist)
#     jaccard_dist_mean.append(jacc_dist)
# jaccard_dist_mean_ind.append(np.mean(jaccard_dist))

In [89]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.to_numpy() ,feature_names = numerical_columns+leave_columns,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3, mode='regression', feature_selection = 'none')

In [90]:
jaccard_dist_mean_lime = []
jaccard_dist_mean_lime_ind = []
k = 50
# for index in [1]:
for index in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    point = X_test.to_numpy()[index]
    xai_functions_dict_lime = {}
    for N in [1000]:
        xai_functions_dict_lime[N] = {}
        for k_ in range(k):
            point = X_test.to_numpy()[index]
            exp = explainer.explain_instance(point, predict_fn, num_features=5, num_samples = N)
            if k_ not in xai_functions_dict_lime[N]:
                xai_functions_dict_lime[N][k_] = [exp]
            else:
                xai_functions_dict_lime[N][k_].append(exp)
    xai_functions = []
    for key in xai_functions_dict_lime[1000]:
        xai_functions.append(xai_functions_dict_lime[1000][key][0])
    comb = list(combinations(xai_functions, 2))
    jaccard_dist = []
    for tup in comb:
        scores1 = get_scores(tup[0], numerical_columns+leave_columns, model_type = "lime")
        scores2 = get_scores(tup[1], numerical_columns+leave_columns, model_type = "lime")
        topk_1 = get_topk_features(scores1, 5)
        topk_2 = get_topk_features(scores2, 5)
        jacc_ind = jaccard_index(topk_1, topk_2)
        jacc_dist = 1-jacc_ind
        jaccard_dist.append(jacc_dist)
        jaccard_dist_mean_lime.append(jacc_dist)
    jaccard_dist_mean_lime_ind.append(np.mean(jaccard_dist))

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

...

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
 /Users/chandan21gupta/miniconda3/envs/xai_surv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning:Function get_feature_names is deprecated; get_featu

In [91]:
print(np.mean(jaccard_dist_mean_lime))
print(np.mean(jaccard_dist_mean_lime_ind))
print(np.mean(jaccard_dist_mean))
print(np.mean(jaccard_dist_mean_ind))

0.45555004859086484
0.4555500485908649
0.4117609329446063
0.4117609329446064


In [ ]:
print(gp[:])
print(gp.parameter_names_flat().tolist())
print(np.atleast_2d(gp[:]))

In [ ]:
display(gp)

In [ ]:
Model: gp_censored_regression
Objective: 1000.6470772672619
Number of Parameters: 17
Number of Optimization Parameters: 17
Updates: True
gp_censored_regression.	value	constraints	priors
sum.Mat52.variance	1.2506505269091814	+ve	
sum.Mat52.lengthscale	(14,)	+ve	
sum.white.variance	0.7197384267986222	+ve	
CensoredGaussian.variance	1.3154504016092228	+ve